In [1]:
%pip install earthengine-api -q
%pip install folium -q

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


In [2]:
import ee

In [3]:
ee.Authenticate()


Successfully saved authorization token.


In [4]:
ee.Initialize()

In [ ]:
import folium

def add_ee_layer(self, ee_image_object, vis_params, name, opacity = 0.5):
    """Adds a method for displaying Earth Engine image tiles to folium map."""
    map_id_dict = ee.Image(ee_image_object).getMapId(vis_params)
    folium.raster_layers.TileLayer(
        tiles=map_id_dict['tile_fetcher'].url_format,
        attr='Map Data &copy; <a href="https://earthengine.google.com/">Google Earth Engine</a>',
        name=name,
        overlay=True,
        control=True,
        opacity=opacity,
    ).add_to(self)
    
folium.Map.add_ee_layer = add_ee_layer

In [ ]:
from IPython.display import display
lat, lon = 45.77, 4.855
band_viz = {
  'min': 0,
  'max': 0.05,
  'palette': ['black', 'blue', 'purple', 'cyan', 'green', 'yellow', 'red']
}

collection = ee.ImageCollection('COPERNICUS/S5P/OFFL/L3_CO').select('CO_column_number_density').filterDate('2022-01-01', '2022-01-02')
point = ee.Geometry.Point((lat, lon))

my_map = folium.Map(location=[lat, lon], zoom_start=10)
my_map.add_ee_layer(collection.mean(), band_viz, 'S5P CO', opacity=0.4)

display(my_map)

In [ ]:
import pandas as pd

def ee_array_to_df(arr, list_of_bands):
    """Transforms client-side ee.Image.getRegion array to pandas.DataFrame."""
    df = pd.DataFrame(arr)

    # Rearrange the header.
    headers = df.iloc[0]
    df = pd.DataFrame(df.values[1:], columns=headers)

    # Remove rows without data inside.
    df = df[['longitude', 'latitude', 'time', *list_of_bands]].dropna()

    # Convert the data to numeric values.
    for band in list_of_bands:
        df[band] = pd.to_numeric(df[band], errors='coerce')

    # Convert the time field into a datetime.
    df['datetime'] = pd.to_datetime(df['time'], unit='ms')

    # Keep the columns of interest.
    df = df[['time','datetime',  *list_of_bands]]

    return df

In [ ]:
def get_ee_data(collection: ee.Collection, position: tuple[float, float], scale: int, bands: list[str]) -> pd.DataFrame:
    point = ee.Geometry.Point(*position)
    data = collection.getRegion(point, scale).getInfo()
    return ee_array_to_df(data, bands)

In [ ]:
df = get_ee_data(collection, (4.8148, 45.7758), 1000, ["CO_column_number_density"])

In [ ]:
def display_ee_data(collection: ee.Collection, position: tuple[float, float], opacity: float = 0.5):
    band_viz = {
        "min": 0,
        "max": 0.05,
        "palette": ["black", "blue", "purple", "cyan", "green", "yellow", "red"],
    }
    my_map = folium.Map(location=[position[0], position[1]], zoom_start=10)
    my_map = my_map.add_ee_layer(collection.mean(), band_viz, "S5P CO", opacity=opacity)
    
    return my_map


In [ ]:
M = display_ee_data(collection, (4.8148, 45.7758), 0.5)

In [ ]:
hm = pd.read_csv("hm.csv")

In [ ]:
latlongs = hm[["lat", "lng"]].dropna().sample(10).values

In [ ]:
scale = 1000
date_start = '2019-01-01'
date_end = '2022-11-05'
to_get = [("COPERNICUS/S5P/OFFL/L3_CH4", ["CH4_column_volume_mixing_ratio_dry_air_bias_corrected"]),
 ("COPERNICUS/S5P/NRTI/L3_CO", ["CO_column_number_density"]),
 ("COPERNICUS/S5P/OFFL/L3_NO2", ["NO2_column_number_density", "NO2_slant_column_number_density"]),
 ("COPERNICUS/S5P/OFFL/L3_CH4", ["CH4_column_volume_mixing_ratio_dry_air_bias_corrected"]),
]


for (lat, lng) in latlongs:
    dfs = []
    for (collection_id, bands) in to_get:
        collection = ee.ImageCollection(collection_id).select(bands).filterDate(date_start, date_end)
        df = get_ee_data(collection, (lat, lng), scale, bands)
        df["lat"] = lat
        df["lng"] = lng

In [ ]:
to_get = [("COPERNICUS/S5P/OFFL/L3_CH4", ["CH4_column_volume_mixing_ratio_dry_air_bias_corrected"]),
 ("COPERNICUS/S5P/NRTI/L3_CO", ["CO_column_number_density"]),
 ("COPERNICUS/S5P/OFFL/L3_NO2", ["NO2_column_number_density", "NO2_slant_column_number_density"]),
 ("COPERNICUS/S5P/OFFL/L3_CH4", ["CH4_column_volume_mixing_ratio_dry_air_bias_corrected"]),
]

position = (4.8148, 45.7758)
scale = 1000
date_start = '2019-01-01'
date_end = '2022-11-05'

dfs = []
for (collection_id, bands) in to_get:
    collection = ee.ImageCollection(collection_id).select(bands).filterDate(date_start, date_end)
    dfs.append(get_ee_data(collection, position, scale, bands))
    
dfs_resampled = []
for tmp_df in dfs:
    df_resampled = tmp_df.resample('D', on='datetime').mean().bfill()
    dfs_resampled.append(df_resampled)
    
combined_datasource = dfs_resampled[0].join(dfs_resampled[1], how='outer').join(dfs_resampled[2], how='outer')

In [ ]:
combined_datasource["CO_column_number_density"].median()

In [11]:
hm = pd.read_csv("hm.csv")

In [16]:
latlongs = hm[["lat", "lng"]].dropna().sample(10).values

In [21]:
scale = 1000
date_start = '2019-01-01'
date_end = '2022-11-05'
to_get = [("COPERNICUS/S5P/OFFL/L3_CH4", ["CH4_column_volume_mixing_ratio_dry_air_bias_corrected"]),
 ("COPERNICUS/S5P/NRTI/L3_CO", ["CO_column_number_density"]),
 ("COPERNICUS/S5P/OFFL/L3_NO2", ["NO2_column_number_density", "NO2_slant_column_number_density"]),
 ("COPERNICUS/S5P/OFFL/L3_CH4", ["CH4_column_volume_mixing_ratio_dry_air_bias_corrected"]),
]


for (lat, lng) in latlongs:
    dfs = []
    for (collection_id, bands) in to_get:
        collection = ee.ImageCollection(collection_id).select(bands).filterDate(date_start, date_end)
        df = get_ee_data(collection, (lat, lng), scale, bands)
        df["lat"] = lat
        df["lng"] = lng

In [47]:
to_get = [("COPERNICUS/S5P/OFFL/L3_CH4", ["CH4_column_volume_mixing_ratio_dry_air_bias_corrected"]),
 ("COPERNICUS/S5P/NRTI/L3_CO", ["CO_column_number_density"]),
 ("COPERNICUS/S5P/OFFL/L3_NO2", ["NO2_column_number_density", "NO2_slant_column_number_density"]),
 ("COPERNICUS/S5P/OFFL/L3_CH4", ["CH4_column_volume_mixing_ratio_dry_air_bias_corrected"]),
]

position = (4.8148, 45.7758)
scale = 1000
date_start = '2019-01-01'
date_end = '2022-11-05'

dfs = []
for (collection_id, bands) in to_get:
    collection = ee.ImageCollection(collection_id).select(bands).filterDate(date_start, date_end)
    dfs.append(get_ee_data(collection, position, scale, bands))
    
dfs_resampled = []
for tmp_df in dfs:
    df_resampled = tmp_df.resample('D', on='datetime').mean().bfill()
    dfs_resampled.append(df_resampled)
    
combined_datasource = dfs_resampled[0].join(dfs_resampled[1], how='outer').join(dfs_resampled[2], how='outer')

In [157]:
combined_datasource["CO_column_number_density"].median()

0.03227181360125718

In [ ]:
def explore_eps(position, scale, duration):
    to_get = [
        ("COPERNICUS/S5P/NRTI/L3_CO",  ["CO_column_number_density"]),
        ("COPERNICUS/S5P/OFFL/L3_NO2", ["NO2_column_number_density", "NO2_slant_column_number_density"]),
        ("COPERNICUS/S5P/OFFL/L3_CH4", ["CH4_column_volume_mixing_ratio_dry_air_bias_corrected"]),
    ]

    date_start, date_end = duration

    dfs = []
    for (collection_id, bands) in to_get:
        collection = ee.ImageCollection(collection_id).select(bands).filterDate(date_start, date_end)
        df = ee_array_to_df(collection.getRegion(ee.Geometry.Point(*position), scale).getInfo(), bands)
        if not df.empty:
            dfs.append(df)

    dfs_resampled = []
    for tmp_df in dfs:
        df_resampled = tmp_df.resample('D', on='datetime').mean().bfill() # <- BFILL FOR INTERPOLATION!
        dfs_resampled.append(df_resampled)

    combined_datasource = []
    if dfs_resampled:
        combined_datasource = dfs_resampled[0]
        for df in dfs_resampled[1:]:
            combined_datasource = combined_datasource.join(df, how='outer')

    return combined_datasource

In [76]:
# IN2022258338BBG 28.6848 77.8173196
explore_eps((28.6848, 77.8173196), 1, ('2022-01-01', '2022-06-22'))

,CO_column_number_density,NO2_column_number_density,NO2_slant_column_number_density
datetime,,,
2022-03-12,NaN,0.000019,0.000107
2022-03-13,NaN,0.000026,0.000144
2022-03-14,NaN,0.000037,0.000172
2022-03-15,NaN,0.000043,0.000148
2022-03-16,0.036134,0.000026,0.000151
...,...,...,...
2022-06-17,0.027313,0.000086,0.000317
2022-06-18,0.026617,0.000086,0.000321
2022-06-19,0.025967,0.000086,0.000341


In [78]:
explore_eps((28.6848, 77.8173196), 10000, ('2022-01-01', '2022-06-22'))


,CO_column_number_density,NO2_column_number_density,NO2_slant_column_number_density
datetime,,,
2022-03-12,NaN,0.000020,0.000110
2022-03-13,NaN,0.000028,0.000150
2022-03-14,NaN,0.000037,0.000170
2022-03-15,NaN,0.000045,0.000151
2022-03-16,0.036038,0.000026,0.000152
...,...,...,...
2022-06-17,0.027466,0.000085,0.000315
2022-06-18,0.026546,0.000085,0.000320
2022-06-19,0.025860,0.000086,0.000343


In [204]:
from IPython.display import display
lat, lon = 45.77, 4.855
band_viz = {
  'min': 0,
  'max': 0.05,
  'palette': ['black', 'blue', 'purple', 'cyan', 'green', 'yellow', 'red']
}

collection = ee.ImageCollection('COPERNICUS/S5P/OFFL/L3_CO').select('CO_column_number_density').filterDate('2022-01-01', '2022-02-22')
point = ee.Geometry.Point((lat, lon))

my_map = folium.Map(location=[lat, lon], zoom_start=10)
my_map.add_ee_layer(collection.median(), band_viz, 'S5P CO', opacity=0.4)

display(my_map)

In [145]:
def explore_eps_map(position, distance, duration):
    to_get = [
        ("COPERNICUS/S5P/NRTI/L3_CO",  ["CO_column_number_density"]),
        ("COPERNICUS/S5P/OFFL/L3_NO2", ["NO2_column_number_density", "NO2_slant_column_number_density"]),
        ("COPERNICUS/S5P/OFFL/L3_CH4", ["CH4_column_volume_mixing_ratio_dry_air_bias_corrected"]),
    ]

    x, y = position
    date_start, date_end = duration

    rectangle = ee.Geometry.Rectangle((x - distance, y - distance, x + distance, y + distance))
    point = ee.Geometry.Point(*position)

    collection_id, bands = to_get[0]
    collection = ee.ImageCollection(collection_id).select(bands).filterBounds(point).filterDate(date_start, date_end)

    print(collection)

    info = collection.getRegion(point, distance).getInfo()
    print(ee_array_to_df(info, bands))

    band_viz = {
      'min': 0,
      'max': 0.05,
      'palette': ['black', 'blue', 'purple', 'cyan', 'green', 'yellow', 'red']
    }
    my_map = folium.Map(location=position, zoom_start=10)
    my_map.add_ee_layer(collection.median(), band_viz, 'S5P CO', opacity=0.4)

    return my_map

In [205]:
map = explore_eps_map((28.6848, 77.8173196), 5, ('2022-01-01', '2022-10-22'))

[['id', 'longitude', 'latitude', 'time', 'CO_column_number_density'], ['20220102T082745_20220102T101506', 28.68480153156563, 77.81730259696293, 1641112060000, None], ['20220104T111245_20220104T130609', 28.68480153156563, 77.81730259696293, 1641294760000, None], ['20220107T115745_20220107T135109', 28.68480153156563, 77.81730259696293, 1641556659000, None], ['20220108T113745_20220108T120847', 28.68480153156563, 77.81730259696293, 1641641859000, None], ['20220109T093745_20220109T113436', 28.68480153156563, 77.81730259696293, 1641721059000, None], ['20220111T104109_20220111T124110', 28.68480153156563, 77.81730259696293, 1641897663000, None], ['20220113T082109_20220113T100910', 28.68480153156563, 77.81730259696293, 1642062063000, None], ['20220115T092609_20220115T112211', 28.68480153156563, 77.81730259696293, 1642238763000, None], ['20220115T110609_20220115T113612', 28.68480153156563, 77.81730259696293, 1642244763000, None], ['20220116T072509_20220116T091140', 28.68480153156563, 77.81730259

In [218]:
to_get = [
        ("COPERNICUS/S5P/NRTI/L3_CO",  ["CO_column_number_density"]),
        ("COPERNICUS/S5P/OFFL/L3_NO2", ["NO2_column_number_density", "NO2_slant_column_number_density"]),
        ("COPERNICUS/S5P/OFFL/L3_CH4", ["CH4_column_volume_mixing_ratio_dry_air_bias_corrected"]),
    ]

x, y = (28.6848, 77.8173196)
date_start, date_end = ('2022-01-01', '2022-10-22')
distance = 5

rectangle = ee.Geometry.Rectangle((x - distance, y + distance, x + distance, y - distance))
point = ee.Geometry.Point(x, y)

collection_id, bands = to_get[0]
collection = ee.ImageCollection(collection_id).filterBounds(point).filterDate(date_start, date_end).select(bands)

In [206]:
info = collection.getInfo()
print(info.keys())
print(info["properties"].keys())

dict_keys(['type', 'bands', 'id', 'version', 'properties', 'features'])
dict_keys(['system:visualization_0_min', 'type_name', 'keywords', 'thumb', 'description', 'source_tags', 'visualization_0_max', 'title', 'product_tags', 'provider', 'visualization_0_min', 'visualization_0_name', 'date_range', 'period', 'system:visualization_0_bands', 'system:is_global', 'provider_url', 'sample', 'tags', 'system:visualization_0_max', 'visualization_0_palette', 'system:visualization_0_palette', 'system:visualization_0_name', 'visualization_0_bands'])


In [215]:
print(rectangle)

ee.Geometry({
  "functionInvocationValue": {
    "functionName": "GeometryConstructors.Polygon",
    "arguments": {
      "coordinates": {
        "constantValue": [
          [
            [
              23.6848,
              72.8173196
            ],
            [
              23.6848,
              82.8173196
            ],
            [
              33.684799999999996,
              82.8173196
            ],
            [
              33.684799999999996,
              72.8173196
            ]
          ]
        ]
      },
      "evenOdd": {
        "constantValue": true
      }
    }
  }
})


In [219]:
band_viz = {
      'min': 0,
      'max': 0.05,
      'palette': ['black', 'blue', 'purple', 'cyan', 'green', 'yellow', 'red']
    }
my_map = folium.Map(location=(x, y), zoom_start=10)
my_map.add_ee_layer(collection.median(), band_viz, 'S5P CO', opacity=0.4)

display(my_map)

In [195]:
print(info["type"], len(info["features"]))
print(info["features"][0].keys())
print(
    info["features"][0]["properties"]["LON_MIN"],
    info["features"][0]["properties"]["LON_MAX"],
    info["features"][0]["properties"]["LAT_MIN"],
    info["features"][0]["properties"]["LAT_MAX"],
)
print(
    info["features"][-1]["properties"]["LON_MIN"],
    info["features"][-1]["properties"]["LON_MAX"],
    info["features"][-1]["properties"]["LAT_MIN"],
    info["features"][-1]["properties"]["LAT_MAX"],
)

for i in info["features"]:
    i = i["properties"]
    print(i["LON_MIN"] < x < i["LON_MAX"], i["LAT_MIN"] < y < i["LAT_MAX"])

print(x, y)

ImageCollection 4647
dict_keys(['type', 'bands', 'id', 'version', 'properties'])
-11.952150344848633 59.686763763427734 51.35420227050781 77.23589324951172
-179.99960327148438 179.99851989746094 64.43506622314453 89.92446899414062
True False
True True
True False
True False
True True
True False
True False
True False
True True
True True
True True
True False
False False
True True
True False
True False
True True
False False
True True
True True
True True
True False
True True
True False
True True
True True
True False
True True
True True
True False
False False
True True
True True
True True
True True
True True
True False
True True
True False
False True
True True
True False
True True
False False
True True
True True
True True
False False
True True
True True
True False
True False
False True
True False
False False
True True
False False
True True
False False
True True
True True
True False
False True
True False
True True
False False
True True
True True
True True
True True
True False
True True
True T